In [2]:
import numpy as np
import pandas as pd

# Candidate Elimination Algorithm
def candidate_elimination(data):
    # Extract features and target labels
    concepts = np.array(data.iloc[:, :-1])
    target = np.array(data.iloc[:, -1])

    # Initialize S and G
    S = concepts[0].copy()
    G = [['?' for _ in range(len(S))] for _ in range(len(S))]  # Initialize G with '?' for all attributes

    for i, instance in enumerate(concepts):
        if target[i] == 'Yes':  # Positive example
            for j in range(len(S)):
                if S[j] != instance[j]:
                    S[j] = '?'  # Generalize S if attribute values don't match
            # Remove hypotheses from G that are inconsistent with the positive example
            G = [g for g in G if all(g[k] == '?' or g[k] == S[k] for k in range(len(S)))]
        else:  # Negative example
            new_G = []
            for j in range(len(S)):
                if S[j] != instance[j]:
                    new_hypothesis = S.copy()
                    new_hypothesis[j] = instance[j]
                    # Add the new hypothesis to new_G if it is consistent with the negative example
                    # and is not more general than any hypothesis in G
                    is_consistent = all(new_hypothesis[k] == '?' or new_hypothesis[k] != concepts[i][k] for k in range(len(S)))
                    is_more_general = any(all(g[k] == '?' or g[k] == new_hypothesis[k] for k in range(len(S))) for g in G)

                    if is_consistent and not is_more_general:
                        new_G.append(new_hypothesis)

            G.extend(new_G) # Add new hypotheses to G

    # Remove hypotheses from G that are more general than any other hypothesis in G
    G = [g for g in G if not any(all(g[k] == '?' or g[k] == h[k] for k in range(len(S))) for h in G if g != h)]

    return S, G # Return the final specific and general hypotheses


# Sample dataset
data = pd.DataFrame(
    [
        ['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same', 'Yes'],
        ['Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same', 'Yes'],
        ['Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change', 'No'],
        ['Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change', 'Yes']
    ],
    columns=['Sky', 'Temp', 'Humidity', 'Wind', 'Water', 'Forecast', 'EnjoySport']
)

# Run Candidate Elimination Algorithm
S_final, G_final = candidate_elimination(data)

# Print results
print("Final Specific Hypothesis (S):", S_final)
print("Final General Hypothesis (G):", G_final)

Final Specific Hypothesis (S): ['Sunny' 'Warm' '?' 'Strong' '?' '?']
Final General Hypothesis (G): [['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?']]
